<h1>Conditional Workflow</h1>

In [37]:
from langgraph.graph import StateGraph, START , END
from typing import TypedDict, Literal

In [38]:
class equation_state(TypedDict):
    a: str
    b: str
    c: str

    discriminant: float
    equation: str
    root: list[float]
    result: str

In [46]:
def get_equation_string(state:equation_state) -> equation_state:
    return {'equation': f"{state['a']}x²{'+' if state['b'] > 0 else '' }{state['b']}x{'+' if state['c'] > 0 else '' }{state['c']}"}

def eval_discriminant(state:equation_state) -> equation_state:
    return {'discriminant': state['b']**2 - (4*state['a']*state['c'])}

def distinct_roots(state:equation_state) -> equation_state:
    root1 = (-state['b'] + (state['discriminant'])**0.5 )/2*state['a']
    root2 = (-state['b'] - (state['discriminant'])**0.5 )/2*state['a']

    return {'root': [root1, root2], 'result' : f"Two distinct roots are \nRoot1 = {root1}\nRoot2 = {root2}"}

def repeated_roots(state:equation_state) -> equation_state:
    root = (-state['b'])/2*state['a']

    return {'root': [root], 'result' : f"Only repeating root is {root}"}

def no_real_roots(state:equation_state) -> equation_state:
    return {'root': [] , 'result' : 'No Real Roots for this equation'}

def conditional_router(state:equation_state) -> Literal['distinct_roots','repeated_roots','no_real_roots']:
    if state['discriminant'] > 0:
        return 'distinct_roots'
    elif state['discriminant'] < 0:
        return 'no_real_roots'
    else:
        return 'repeated_roots'

In [47]:
graph = StateGraph(equation_state)

graph.add_node('get_equation_string',get_equation_string)
graph.add_node('eval_discriminant',eval_discriminant)
graph.add_node('distinct_roots',distinct_roots)
graph.add_node('repeated_roots',repeated_roots)
graph.add_node('no_real_roots',no_real_roots)

graph.add_edge(START, 'get_equation_string')
graph.add_edge('get_equation_string', 'eval_discriminant')

graph.add_conditional_edges('eval_discriminant',conditional_router)
graph.add_edge('distinct_roots',END)
graph.add_edge('repeated_roots',END)
graph.add_edge('no_real_roots',END)

workflow = graph.compile()

In [55]:
initial_state = {'a':2, 'b':10, 'c':2}

print(workflow.invoke(initial_state)['result'])

Two distinct roots are 
Root1 = -0.8348486100883203
Root2 = -19.165151389911678
